In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from abg_python.snapshot_utils import openSnapshot
from abg_python.cosmo_utils import load_AHF
from itk import inrange, loadpickle, pickle_save_dict, n_array_equal, sync_lim
from numba import njit
from tqdm import tqdm
from glob import glob
from os import path
%matplotlib inline

In [ ]:
exec(open('scripts/halo_analysis_scripts.py').read())

In [ ]:
fig, axes = plt.subplots(4, 6, sharex=True, sharey='row', gridspec_kw={'wspace': .05, 'hspace':.03}, figsize=[4.8*6,4.8*4], dpi=150)
cmaps = {'radfboff':plt.cm.Blues, 'good':plt.cm.Greens, 'CRsoff_veryhighradfb':plt.cm.Reds, 'veryslowwinds':plt.cm.Purples, 'CRsoff_veryfastwinds':plt.cm.Greys}
indices = {'radfboff':1, 'good':0, 'CRsoff_veryhighradfb':2, 'veryslowwinds':3, 'CRsoff_veryfastwinds':4}
#names = {'radfboff':'Radiative feedback off', 'good':'Good model', 'CRsoff_veryhighradfb':'', 'veryslowwinds':3, 'CRsoff_veryfastwinds':4}

for model, simdir in Frontera_sims['push']['h206'].items():
    if model=='CRsoff':
        continue
    simname = path.basename(simdir)
    print(f'\nComputing sim {simname}.')
    i = indices[model]
    
    # Load profiles for all snapshots in each redshift bin
    allprofiles = loadpickle(f'data/{simname}_allprofiles_widezbins.pkl')['allprofiles']
    
    # For each redshift bin, plot median rho, T, P_thermal, and e_CR profiles
    plot_profiles_zbins(allprofiles, axes[0,0], cmap=cmaps[model], ylabel=True)
    plot_profiles_zbins(allprofiles, axes[0,i+1], cmap=cmaps[model])

    plot_profiles_zbins(allprofiles, axes[1,0], 'T lin', cmap=cmaps[model], ylabel=True)
    plot_profiles_zbins(allprofiles, axes[1,i+1], 'T lin', cmap=cmaps[model])

    plot_profiles_zbins(allprofiles, axes[2,0], 'P_th lin', cmap=cmaps[model], ylabel=True)
    plot_profiles_zbins(allprofiles, axes[2,i+1], 'P_th lin', cmap=cmaps[model])

    plot_profiles_zbins(allprofiles, axes[3,0], 'P_CR lin', cmap=cmaps[model], ylabel=True, xlabel=True)
    plot_profiles_zbins(allprofiles, axes[3,i+1], 'P_CR lin', cmap=cmaps[model], xlabel=True)
    
    axes[0,i+1].legend()
    axes[0,i+1].set_title(model)

sync_lim(axes[2:4, 0], 'y')
# plt.savefig(f'Figures/rho_Tlin_Pthermallin_PCRlin_profile_{"h206_push_differentmodels"}_AGN_widezbins.pdf')